In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
emdat = pd.read_csv('emdat.csv')

In [2]:
emdat.head()  

,Disaster Subgroup,Disaster Type,Disaster Subtype,ISO,Country,Subregion,Region,Latitude,Longitude,Start Year,Total Deaths
0,Meteorological,Storm,Tropical cyclone,USA,United States of America,Northern America,Americas,NaN,NaN,1900,6000.0
1,Hydrological,Flood,Flood (General),JAM,Jamaica,Latin America and the Caribbean,Americas,NaN,NaN,1900,300.0
2,Biological,Epidemic,Viral disease,JAM,Jamaica,Latin America and the Caribbean,Americas,NaN,NaN,1900,30.0
3,Geophysical,Volcanic activity,Ash fall,JPN,Japan,Eastern Asia,Asia,NaN,NaN,1900,30.0
4,Geophysical,Earthquake,Ground movement,TUR,Türkiye,Western Asia,Asia,40.3,43.1,1900,140.0


In [22]:
def analyze_dominant_disaster_by_country(file_path, top_n=10):

    try:
           df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"エラー: ファイルが見つかりません: {file_path}")
        return
    
    df_clean = df.dropna(subset=['Country', 'Disaster Type'])
    
    disaster_counts = df_clean.groupby(['Country', 'Disaster Type']).size().reset_index(name='Count')

    idx = disaster_counts.groupby('Country')['Count'].idxmax()
    dominant_disasters = disaster_counts.loc[idx]
 
    total_country_events = df_clean['Country'].value_counts().reset_index(name='Total Events')
    total_country_events.columns = ['Country', 'Total Events']

    result_df = pd.merge(dominant_disasters, total_country_events, on='Country')
    result_df = result_df.sort_values(by='Total Events', ascending=False)
    
    print(f"\n 国別 - 最も発生件数が多い災害の種類 (上位 {top_n} か国)")
    print("--------------------------------------------------")
    
    display_cols = ['Country', 'Disaster Type', 'Count', 'Total Events']
    display_df = result_df[display_cols].head(top_n)
    
    display_df.columns = ['国名', '最も多い災害', '当該件数', '全災害合計件数']
    
    print(display_df.to_string(index=False))
    
file_name = 'emdat.csv'
analyze_dominant_disaster_by_country(file_name)


 国別 - 最も発生件数が多い災害の種類 (上位 10 か国)
--------------------------------------------------
                      国名 最も多い災害  当該件数  全災害合計件数
United States of America  Storm   742     1172
                   China  Storm   331     1023
                   India  Flood   327      785
             Philippines  Storm   400      710
               Indonesia  Flood   275      636
                   Japan  Storm   198      391
              Bangladesh  Storm   187      368
                  Mexico  Storm   131      311
                  Brazil  Flood   185      289
                Viet Nam  Storm   139      272


stoom(嵐),fiood(洪水)が多く